In [4]:
import os
import sys
import uuid

pipeline_dir = os.path.join(os.getcwd(), '..')
if pipeline_dir not in sys.path:
    sys.path.append(pipeline_dir)

In [5]:
from pipeline import Pipeline, events, functions

# Functions
## Defining Functions
You can define a lambda function by declaring a method inside your `pipeline.Pipeline` object.  The method must be decorated with an event decorator to be considered a decorator.

In [6]:
class MyPipeline(Pipeline):
    
    def __init__(self):
        super().__init__()
    
    @events.invoke
    def lambda_func(self, event, context):
        print(event)

pipeline = MyPipeline()

You can use arguments to customize more complex events:

In [7]:
@events.http(method="get", path="todos", cors="true")
def lambda_func_http(self, event, context):
    print(event)

You can customize your function's runtime configuration with more decorators:

In [8]:
@events.invoke
@functions.timeout(30) # Specify function's timeout in seconds
@functions.memory(3008) # Specify function's memory limit in MB
def lambda_func(self, event, context):
    print(event)

## Writing Functions
#### Input Handling
The event decorator used by the pipeline method modifies the input event message.  For example, the `HTTP POST` decorator modifies the original message by calling `json.loads(event['body'])` and passing the resulting dictionary to your pipeline method.  This feature is intended for QOL, as input messaging parsing happens automatically and lambda functions.  You may use the `legacy` input parameter to disable this behavior.

In [9]:
@events.http(method="post", path="todos", cors="true")
def lambda_func(self, event, contet):
    assert type(event) == dict

@events.http(method="post", path="todos", cors="true", legacy=True)
def lambda_func(self, event, contet):
    assert type(event) == str

#### Output Handling
The output of your lambda function is determined by your pipeline method's return statement.  Please be careful to ensure that functions are returning the appropriate output for the event type.  For example, any HTTP response must be of the form `json.dumps({'statusCode': 200, 'body': 'return_message'})`

Not all functions require a return statement.  A lambda function which simply relays a message to a SNS Topic does not have a meaningful return other than to confirm the message was sent succesfully.

## Helper Functions
Only methods decorated with an event trigger are considered lambda functions.  You may define other helper methods used by your pipeline:

In [12]:
class MyPipeline(Pipeline):
    
    def __init__(self):
        super().__init__()
    
    def helper_function(self):
        return "Hello World!"
    
    @events.invoke
    def lambda_func(self, event, context):
        response = self.helper_function()
        print(response)

## Deploying Functions
AWS Lambda forbids declaring a class function as a handler.  This means we need to add entrypoints to the bottom of our python file.  Make sure that the declared variable has the same name as your function.  This will allow your functions to be deployed succesfully.

In [10]:
lambda_func = pipeline.lambda_func